# CSV Writer - Comma-separated values

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys
import time

for path in ['python', '.', '..']:
    readersPath = os.path.join(path, 'core')
    if readersPath not in sys.path:
        sys.path.extend([readersPath])

import csv
from datetime import datetime

import unittest

if __name__ == '__main__':
    from track import Track

from base_writer import BaseWriter

## Main Class

In [2]:
class CsvWriter(BaseWriter):
    '''CSV file - Comma-separated values'''

    def __init__(self, filename=None, track=None):
        '''Basic init just records the filename'''

        super().__init__(filename, track)


    def prepare(self, track):
        '''Prepare CSV prior to being saved'''

        self.buffer = []

        # Basic header
        header = []
        for field in track.data:
            if field == 'ts':
                header.append('date')
                header.append('time')
            else:
                header.append(field)

        self.buffer.append(header)

        # Determine field formats
        formats = self.getFormats(track)

        # Convert records to CSV
        for i in range(track.numRecords):
            record = []

            for field in track.data:
                if field == 'ts':
                    dt = datetime.fromtimestamp(track.data[field][i])
                    record.append(dt.strftime('%Y-%m-%d'))
                    value = dt.strftime('%H:%M:%S.%f')[:-3]
                elif field in formats:
                    value = formats[field].format(track.data[field][i])
                else:
                    value = track.data[field][i]

                record.append(value)

            self.buffer.append(record)


    def save(self, filename=None):
        '''Save GPX to disk'''
        
        if filename:
            self.filename = filename
            
        with open(self.filename, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(self.buffer)

## Unit Tests

In [3]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    sbnFilename = os.path.join(projdir, 'sessions', '20071227', 'MIKE_G_1003053_20071227_165512_DLG.SBP')
    sbnTrack = Track(sbnFilename)

    csvFilename = os.path.join(projdir, 'sessions', 'csv_writer.csv')
    csvWriter = CsvWriter(csvFilename)

    pc1 = time.perf_counter()
    sbnTrack.load()
    csvWriter.prepare(sbnTrack)
    pc2 = time.perf_counter()

    print("\nTest files loaded and prepared in %0.2f seconds" % (pc2 - pc1))


Test files loaded and prepared in 0.04 seconds


In [4]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)


----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK
